In [89]:
import requests
import pandas as pd

api_key = 'ITL1B9MC4L5G7SEN'
symbol = 'SIE.DE'
output_size = 'full'

url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize={output_size}&apikey={api_key}'
r = requests.get(url)
time_series = r.json()['Time Series (Daily)']

data = []
for date, values in time_series.items():
    data.append([
        date,
        float(values["1. open"]),
        float(values["2. high"]),
        float(values["3. low"]),
        float(values["4. close"]),
        int(values["5. volume"])
    ])

df0 = pd.DataFrame(data, columns=["date", "open", "high", "low", "close", "volume"])
print(df0.shape)
df0.head(15)

(4943, 6)


,date,open,high,low,close,volume
0,2024-06-14,170.80,171.10,165.68,165.74,1686923
1,2024-06-13,177.28,177.56,170.44,170.84,1395794
2,2024-06-12,173.12,178.58,172.30,178.10,975006
3,2024-06-11,174.70,175.44,172.18,172.46,953042
4,2024-06-10,174.12,174.52,172.06,174.44,717411
5,2024-06-07,175.00,175.26,172.62,174.44,1034765
6,2024-06-06,178.88,179.52,175.14,175.68,826573
7,2024-06-05,175.60,178.82,175.38,178.82,828300
8,2024-06-04,178.58,179.00,174.28,174.28,1232510
9,2024-06-03,178.80,179.60,177.10,178.16,745486


In [98]:
import pandas_ta as pta
import ta

df = df0.copy()

df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

df['RSI'] = pta.rsi(df['close'], length=11)
df['William%R'] = pta.willr(df['high'], df['low'], df['close'], length=11)
df['WMA'] = pta.wma(df['close'], length=11)
df['EMA'] = pta.ema(df['close'], length=11)
df['SMA'] = pta.sma(df['close'], length=11)
df['HMA'] = pta.hma(df['close'], length=11)
df['CCI'] = pta.cci(df['high'], df['low'], df['close'], length=11)
df['CMO'] = pta.cmo(df['close'], length=11)

macd = pta.macd(df['close'])
df['MACD'] = macd.iloc[:, 0] 
df['MACD_signal'] = macd.iloc[:, 1] 

ppo = pta.ppo(df['close'])
df['PPO'] = ppo.iloc[:, 0] 
df['PPO_signal'] = ppo.iloc[:, 1]

df['ROC'] = pta.roc(df['close'], length=11)
df['CMFI'] = ta.volume.ChaikinMoneyFlowIndicator(high=df['high'], low=df['low'], close=df['close'], volume=df['volume'], window=11).chaikin_money_flow()

dmi = pta.adx(df['high'], df['low'], df['close'], length=11)
df['DMI'] = dmi['ADX_11']
df['DI+'] = dmi['DMP_11']
df['DI-'] = dmi['DMN_11']

df['PSI'] = pta.pvt(df['close'], df['volume'])
df['TripleEMA'] = pta.ema(df['close'], length=11)

def label_data(data, window_size=11):
    data['Label'] = 'Hold'
    for i in range(window_size, len(data) - window_size):
        window = data['Close'].iloc[i-window_size:i+window_size]
        min_index = window.idxmin()
        max_index = window.idxmax()
        if i == min_index:
            data.at[i, 'Label'] = 'Buy'
        elif i == max_index:
            data.at[i, 'Label'] = 'Sell'
    return data

df = df.dropna()

for column in df.columns:
    df[column] = (df[column] - df[column].mean()) / df[column].std()

print(df.info())
df.head(15)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4910 entries, 2024-04-29 to 2005-01-03
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   open         4910 non-null   float64
 1   high         4910 non-null   float64
 2   low          4910 non-null   float64
 3   close        4910 non-null   float64
 4   volume       4910 non-null   float64
 5   RSI          4910 non-null   float64
 6   William%R    4910 non-null   float64
 7   WMA          4910 non-null   float64
 8   EMA          4910 non-null   float64
 9   SMA          4910 non-null   float64
 10  HMA          4910 non-null   float64
 11  CCI          4910 non-null   float64
 12  CMO          4910 non-null   float64
 13  MACD         4910 non-null   float64
 14  MACD_signal  4910 non-null   float64
 15  PPO          4910 non-null   float64
 16  PPO_signal   4910 non-null   float64
 17  ROC          4910 non-null   float64
 18  CMFI         4910 non-null   f

,open,high,low,close,volume,RSI,William%R,WMA,EMA,SMA,...,MACD_signal,PPO,PPO_signal,ROC,CMFI,DMI,DI+,DI-,PSI,TripleEMA
date,,,,,,,,,,,,,,,,,,,,,
2024-04-29,3.076558,3.043858,3.092136,3.050147,-1.005469,0.029870,-0.774977,3.092911,3.082873,3.160439,...,-3.092522,0.725794,0.310551,-0.867154,0.593130,0.008023,-0.072047,0.635317,-1.165691,3.082873
2024-04-26,2.963077,3.014868,2.982005,3.049468,-0.845337,0.026385,-0.779657,3.072712,3.076234,3.136978,...,-2.766793,0.528790,-0.156545,-0.661845,1.416329,0.045257,-0.298816,1.321821,-1.165778,3.076234
2024-04-25,2.951525,2.913741,2.909497,2.916455,-0.787360,-0.594655,-1.350263,3.034277,3.048549,3.093012,...,-2.953416,0.368749,-0.454217,-1.235735,1.475481,0.143473,-0.585098,1.442051,-1.185216,3.048549
2024-04-24,2.960359,2.969024,2.973113,2.948351,-0.795461,-0.399684,-0.967255,3.008489,3.030790,3.050646,...,-2.833950,0.155636,-0.801487,-1.187467,0.676465,0.132887,-0.206910,1.148627,-1.180532,3.030790
2024-04-23,2.948807,2.934641,2.960116,2.972104,-0.841508,-0.252206,-0.693016,2.993725,3.019946,3.023737,...,-2.552584,-0.059429,-1.083316,-0.766367,0.770860,0.136656,-0.335572,1.053553,-1.177409,3.019946
2024-04-22,2.959679,2.940708,2.962168,2.927313,-0.921940,-0.473476,-0.880974,2.975991,3.003450,3.000461,...,-2.432491,-0.216458,-1.189439,-0.670062,0.627685,0.128352,-0.407860,0.817596,-1.182139,3.003450
2024-04-19,2.894445,2.856436,2.906761,2.891346,-0.632314,-0.645725,-1.228159,2.956152,2.983713,2.978355,...,-2.381552,-0.329235,-1.183340,-0.640788,0.321716,0.181649,-0.530909,1.185626,-1.189275,2.983713
2024-04-18,2.982104,2.981833,2.992266,2.978211,-0.803356,-0.062907,0.134007,2.954467,2.981733,2.966532,...,-1.916307,-0.381294,-1.053601,-0.336813,0.049734,0.004562,0.568181,0.756493,-1.176683,2.981733
2024-04-17,2.896483,2.903628,2.923178,2.881845,-0.973529,-0.539043,-1.303704,2.938706,2.964034,2.951322,...,-1.889947,-0.458987,-1.002525,-0.442311,-0.282494,-0.071582,0.311598,1.178151,-1.185284,2.964034


In [99]:
def label_data(prices, window_size=11):
    labels = ['hold'] * len(prices)
    half_window = window_size // 2
    
    for i in range(half_window, len(prices) - half_window):
        window = prices[i - half_window: i + half_window + 1]
        if prices[i] == max(window):
            labels[i] = "sell"
        elif prices[i] == min(window):
            labels[i] = "buy"
    
    return labels


df['label'] = label_data(df['close'])

df.sample(15)

/tmp/ipykernel_68178/2464335656.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if prices[i] == max(window):
/tmp/ipykernel_68178/2464335656.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif prices[i] == min(window):


,open,high,low,close,volume,RSI,William%R,WMA,EMA,SMA,...,PPO,PPO_signal,ROC,CMFI,DMI,DI+,DI-,PSI,TripleEMA,label
date,,,,,,,,,,,,,,,,,,,,,
2023-09-28,1.590431,1.582907,1.582454,1.622971,-0.794343,0.613934,0.104630,1.583153,1.568358,1.591816,...,1.168885,0.827666,0.275180,-0.040992,0.054893,0.944766,-0.070813,-1.393186,1.568358,hold
2014-12-03,-0.099238,-0.116370,-0.079675,-0.105283,-0.480507,0.326648,1.447449,-0.132995,-0.132821,-0.152355,...,-0.046362,0.437520,0.022963,-0.335380,-0.623801,-0.762803,-0.117373,-0.025555,-0.132821,hold
2011-01-17,-0.283448,-0.290083,-0.269862,-0.262402,-0.609769,-0.610502,-0.953794,-0.244065,-0.238129,-0.228346,...,-0.336201,-0.241799,-0.593782,-0.069306,-1.002082,-0.498455,0.728879,1.322811,-0.238129,hold
2014-11-24,-0.097095,-0.105735,-0.080290,-0.093048,-0.311759,0.202653,0.117715,-0.077833,-0.089569,-0.088825,...,0.413819,0.539986,0.311197,0.177876,-1.074365,-0.423553,-0.132769,-0.024391,-0.089569,hold
2010-03-19,-0.847365,-0.859871,-0.856946,-0.864707,3.488141,-0.583386,-0.940325,-0.828904,-0.825496,-0.805761,...,0.101542,-0.489511,-0.813828,0.995682,-0.710726,-0.530733,0.402708,0.009491,-0.825496,hold
2005-07-01,-1.188386,-1.186138,-1.167966,-1.175642,0.163895,-1.019027,-0.256712,-1.161972,-1.149812,-1.148218,...,-0.753081,-1.195008,-0.692216,0.981724,1.051018,-1.376563,2.392457,0.279712,-1.149812,hold
2021-08-12,1.841856,1.853254,1.846494,1.861173,-0.814103,0.399679,1.297093,1.812236,1.817500,1.785600,...,-0.441598,0.200674,0.240831,0.699274,-0.879752,0.279144,0.170238,-1.226598,1.817500,sell
2015-11-04,-0.138444,-0.147973,-0.129931,-0.155754,-0.503439,-0.132035,-0.834212,-0.141697,-0.132660,-0.135703,...,-0.233031,-1.526695,-0.599608,-0.673556,-1.171921,-0.269565,-0.410996,0.217797,-0.132660,hold
2020-07-16,0.418993,0.442148,0.409936,0.456317,-0.593135,-0.341036,-0.135244,0.494617,0.488266,0.488502,...,-0.357575,0.403252,0.615275,0.293648,-0.625467,-0.619339,0.678149,-1.499324,0.488266,hold
